In [1]:
!pip install --upgrade category_encoders

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.linear_model import Ridge
from category_encoders import TargetEncoder
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline

# 1.1 DATA IMPORT AND FINDING CONTINOUS AND CATEGORICAL

In [18]:
dataset = fetch_openml("credit-g", as_frame=True)

C:\Users\tamim\anaconda3\lib\site-packages\sklearn\datasets\_openml.py:417: UserWarning: Multiple active versions of the dataset matching the name credit-g exist. Versions may be fundamentally different, returning version 1.
  warn(


In [19]:
type(dataset.data)

pandas.core.frame.DataFrame

In [20]:
dataset.data

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
0,<0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4.0,male single,none,4.0,real estate,67.0,none,own,2.0,skilled,1.0,yes,yes
1,0<=X<200,48.0,existing paid,radio/tv,5951.0,<100,1<=X<4,2.0,female div/dep/mar,none,2.0,real estate,22.0,none,own,1.0,skilled,1.0,none,yes
2,no checking,12.0,critical/other existing credit,education,2096.0,<100,4<=X<7,2.0,male single,none,3.0,real estate,49.0,none,own,1.0,unskilled resident,2.0,none,yes
3,<0,42.0,existing paid,furniture/equipment,7882.0,<100,4<=X<7,2.0,male single,guarantor,4.0,life insurance,45.0,none,for free,1.0,skilled,2.0,none,yes
4,<0,24.0,delayed previously,new car,4870.0,<100,1<=X<4,3.0,male single,none,4.0,no known property,53.0,none,for free,2.0,skilled,2.0,none,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,no checking,12.0,existing paid,furniture/equipment,1736.0,<100,4<=X<7,3.0,female div/dep/mar,none,4.0,real estate,31.0,none,own,1.0,unskilled resident,1.0,none,yes
996,<0,30.0,existing paid,used car,3857.0,<100,1<=X<4,4.0,male div/sep,none,4.0,life insurance,40.0,none,own,1.0,high qualif/self emp/mgmt,1.0,yes,yes
997,no checking,12.0,existing paid,radio/tv,804.0,<100,>=7,4.0,male single,none,4.0,car,38.0,none,own,1.0,skilled,1.0,none,yes
998,<0,45.0,existing paid,radio/tv,1845.0,<100,1<=X<4,4.0,male single,none,4.0,no known property,23.0,none,for free,1.0,skilled,1.0,yes,yes


In [21]:
continous_variables = dataset.data.select_dtypes("number")
categorical_variables = dataset.data.select_dtypes("category")

In [22]:
continous_variables 

,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents
0,6.0,1169.0,4.0,4.0,67.0,2.0,1.0
1,48.0,5951.0,2.0,2.0,22.0,1.0,1.0
2,12.0,2096.0,2.0,3.0,49.0,1.0,2.0
3,42.0,7882.0,2.0,4.0,45.0,1.0,2.0
4,24.0,4870.0,3.0,4.0,53.0,2.0,2.0
...,...,...,...,...,...,...,...
995,12.0,1736.0,3.0,4.0,31.0,1.0,1.0
996,30.0,3857.0,4.0,4.0,40.0,1.0,1.0
997,12.0,804.0,4.0,4.0,38.0,1.0,1.0
998,45.0,1845.0,4.0,4.0,23.0,1.0,1.0


In [23]:
categorical_variables.columns

Index(['checking_status', 'credit_history', 'purpose', 'savings_status',
       'employment', 'personal_status', 'other_parties', 'property_magnitude',
       'other_payment_plans', 'housing', 'job', 'own_telephone',
       'foreign_worker'],
      dtype='object')

In [24]:
data = dataset.data.drop(['checking_status','credit_history', 'purpose', 'savings_status', 'employment','personal_status', 'other_parties', 'property_magnitude', 'other_payment_plans','housing', 'job','own_telephone','foreign_worker'], axis = 'columns')

In [25]:
data

,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents
0,6.0,1169.0,4.0,4.0,67.0,2.0,1.0
1,48.0,5951.0,2.0,2.0,22.0,1.0,1.0
2,12.0,2096.0,2.0,3.0,49.0,1.0,2.0
3,42.0,7882.0,2.0,4.0,45.0,1.0,2.0
4,24.0,4870.0,3.0,4.0,53.0,2.0,2.0
...,...,...,...,...,...,...,...
995,12.0,1736.0,3.0,4.0,31.0,1.0,1.0
996,30.0,3857.0,4.0,4.0,40.0,1.0,1.0
997,12.0,804.0,4.0,4.0,38.0,1.0,1.0
998,45.0,1845.0,4.0,4.0,23.0,1.0,1.0


In [26]:
data.shape

(1000, 7)

In [27]:
dataset.target

0      good
1       bad
2      good
3      good
4       bad
       ... 
995    good
996    good
997    good
998     bad
999    good
Name: class, Length: 1000, dtype: category
Categories (2, object): ['good', 'bad']

In [28]:
dataset.target.value_counts(normalize=True)

good    0.7
bad     0.3
Name: class, dtype: float64

In [29]:
dataset.target

0      good
1       bad
2      good
3      good
4       bad
       ... 
995    good
996    good
997    good
998     bad
999    good
Name: class, Length: 1000, dtype: category
Categories (2, object): ['good', 'bad']

#  1.2 SPLITTING THE DATASET INTO TRANING AND TESTING 

In [32]:
X= data

y= dataset.target

In [33]:
y= y.astype('category').cat.codes

In [34]:
# FOR TRAINING AND TESTING
X_train, X_test, y_train, y_test = train_test_split(X, y)
print(y_train)

245    0
428    0
13     1
759    1
797    0
      ..
335    1
178    0
0      0
841    0
561    1
Length: 750, dtype: int8


In [35]:
# FOR VALIDATION
X_train, X_val, y_train, y_val =  train_test_split(X, y)

In [37]:
## EVALAUTION AND PREPROCESSING

In [39]:
## WITH KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Predicting results using Test data set
pred = knn.predict(X_val)
from sklearn.metrics import accuracy_score
accuracy_score(pred,y_val)

0.624

In [40]:
## WITH  LogisticRegression

Logreg = LogisticRegression()
Logreg.fit(X_train, y_train)

# Predicting results using Test data set
pred = Logreg.predict(X_val)

from sklearn.metrics import accuracy_score
accuracy_score(pred,y_val)

0.724

# 1.3 USING COLUMN TRANSFER AND PIPELINE

In [41]:
## ENCODING CATEGORICAL AND CONTINOUS DATA

numeric_features = ["installment_commitment"]
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_features = ["num_dependents"]
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [42]:
## MAKING PIPELINE

categorical = data.dtypes == object

preprocess = make_column_transformer(
    (StandardScaler(), ~categorical),
    (OneHotEncoder(), categorical))

model_pipeline_knn = make_pipeline(preprocess, KNeighborsClassifier(n_neighbors = 5))
model_pipeline_knn.fit(X_train, y_train)

score_model_pipeline_knn = cross_val_score(model_pipeline_knn, X_val, y_val, cv = 5, scoring = 'accuracy')

print('mean score_model_pipeline_knn: {:.3f}'.format(np.mean(score_model_pipeline_knn)))

mean score_model_pipeline_knn: 0.688


# 1.4 Fine tuning with GridSerachCV 

In [43]:
## for KNeighborsClassifier()

knn = KNeighborsClassifier()

pipe = Pipeline(steps=[('preprocess', preprocess), ('knn', knn)])
param_grid = {
    'knn__n_neighbors': [2,5,15, 30, 45, 64]
}

grid = GridSearchCV(pipe, param_grid, cv=10, scoring='accuracy')

grid.fit(X_val, y_val)

grid.best_params_
#{'knn__n_neighbors': 5}
grid.best_score_

0.724

In [44]:
print("test dataset score: {:.3f}".format(grid.score(X_test, y_test)))

test dataset score: 0.692


In [45]:
## for LogisticRegression()

pipeline = Pipeline(steps=[
     ('preprocess', preprocess),
     ('model', Ridge())
])

# Define the parameter grid to be used in GridSearch
param_grid = {'model__alpha': np.arange(1, 30, 2)}

search = GridSearchCV(pipeline, param_grid)
search.fit(X_val, y_val)
search.best_score_

-0.03164469038303046

In [46]:
print("test dataset score: {:.3f}".format(search.score(X_test, y_test)))

test dataset score: 0.074


In [47]:
## Changing the cross validation value 

from sklearn.pipeline import make_pipeline
categorical = data.dtypes == object

preprocess = make_column_transformer(
    (StandardScaler(), ~categorical),
    (OneHotEncoder(), categorical))

model_pipeline_knn = make_pipeline(preprocess, KNeighborsClassifier(n_neighbors = 10))
model_pipeline_knn.fit(X_train, y_train)

score_model_pipeline_knn = cross_val_score(model_pipeline_knn, X_val, y_val, cv = 10, scoring = 'accuracy')

print('mean score_model_pipeline_knn: {:.3f}'.format(np.mean(score_model_pipeline_knn)))

mean score_model_pipeline_knn: 0.700


In [48]:
## for KNeighborsClassifier()

knn = KNeighborsClassifier()

pipe = Pipeline(steps=[('preprocess', preprocess), ('knn', knn)])
param_grid = {
    'knn__n_neighbors': [2,5,15, 30, 45, 64]
}
grid = GridSearchCV(pipe, param_grid, cv=10, scoring='accuracy')

grid.fit(X_val, y_val)

grid.best_params_
#{'knn__n_neighbors': 5}
grid.best_score_

0.724

In [49]:
print("test dataset score: {:.3f}".format(grid.score(X_test, y_test)))

test dataset score: 0.692


In [50]:
## for LogisticRegression()


# Pay attention to the name of the second step, i. e. 'model'
pipeline = Pipeline(steps=[
     ('preprocess', preprocess),
     ('model', Ridge())
])

# Define the parameter grid to be used in GridSearch
param_grid = {'model__alpha': np.arange(1, 30, 2)}

search = GridSearchCV(pipeline, param_grid)
search.fit(X_val, y_val)
search.best_score_

-0.03164469038303046

In [51]:
print("test dataset score: {:.3f}".format(search.score(X_test, y_test)))

test dataset score: 0.074


In [52]:
search.fit(X_test, y_test)

GridSearchCV(estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('standardscaler',
                                                                         StandardScaler(),
                                                                         duration                  True
credit_amount             True
installment_commitment    True
residence_since           True
age                       True
existing_credits          True
num_dependents            True
dtype: bool),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(),
                                                                         duration                  False
credit_amount             False
installment_commitment    False
residence_since           False
age                       False
existing_credits          False
num_dependents 

# 1.5 Parameter Visualization

In [54]:
# For KNearestNeighbors
knn_c_values = range(10, 20)
knn_mean_test_scores = np.asarray(grid.cv_results_['mean_test_score'])
knn_mean_train_scores = np.asarray(grid.cv_results_['mean score_model_pipeline_knn'])
std_test = np.asarray(grid.cv_results_['std_test_score'])
std_train = np.asarray(grid.cv_results_['std_train_score'])

fig, ax = plt.subplots(figsize=(15,10))

KeyError: 'mean score_model_pipeline_knn'